In [72]:
#------------------------ Imports ------------------------#
import imutils
import cv2
import numpy as np
import time
#------------------------ Videos festlegen ------------------------#
Video = 3
Videos = {
            1:["Simulation1",(917,518),170,20,13],
            2:["3D/Simulation1_Front",(962,548),200,20,48],
            3:["3D/Simulation2_Front",(962,548),200,20,48],
            4:["Kosmos1",(790,400),450,50,0],
            5:["Kosmos2",(790,400),450,50,0]
         }

#------------------------ Videoparameter festlegen ------------------------#
vs = cv2.VideoCapture(Videos[Video][0]+".mp4")
center = Videos[Video][1]
rotor_radius = Videos[Video][2]
hub_circle_radius = Videos[Video][3]
img_count = Videos[Video][4]

#------------------------ Kreis fürs Rotor ausblenden ------------------------#
frames_per_rotation = 180
theta = np.linspace(0, -2*np.pi, num=frames_per_rotation) #180 Werte zwischen 0 und 2*pi
x = rotor_radius*np.sin(theta) # x-Koordinaten nach Einheitskreis
y = rotor_radius*np.cos(theta) # y-Koordinaten nach Einheitskreis
rotor_positions = [(int(x[i])+center[0],int(y[i])+center[1]) for i in range(frames_per_rotation)]

#------------------------ Variablen ------------------------#
frame_old = None
mid_old = None
p_lst = []
p_lst2 = []
flag = 0

#------------------------ Algorithmus ------------------------#
for i in range(0,int(vs.get(cv2.CAP_PROP_FRAME_COUNT))):
    img_count = (img_count+1)%frames_per_rotation            # Counter hochzählen (maximal 180)
    ret, frame = vs.read()                                   # Bild aus Video einlesen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)           # Bild zu grau konvertieren
    gray = cv2.GaussianBlur(gray, (21, 21), 0)               # Bild blurren
        
    if frame_old is None:                                          # Erstes Bild initialisieren
        frame_old = gray                                           #
        continue                                             #
    
    line_image = np.zeros(frame.shape[:-1])                  # Neues leeres Bild mit gleicher Größe erstellen (Für spätere Berechnungen)
    for i in range(0,frames_per_rotation,frames_per_rotation//3):
        pos = rotor_positions[(img_count+i)%frames_per_rotation]
        cv2.line(line_image, pos, center, (255,255,255), 65)              # Angewinkelte Linie auf leeres Bild einzeichnen
        #cv2.line(frame, pos, center, (255,255,255), 65)                  # Diese Linie auch auf frame anzeigen (optional)
    
    
    frameDelta = cv2.absdiff(frame_old, gray)                                                   # Differenz (Delta) zwischen altem und neuem Bild berechnen
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1]                      # Delta threshholde
    thresh = cv2.dilate(thresh, None, iterations=4)                                       #
    thresh = thresh - np.uint8(line_image)
    thresh = cv2.threshold(thresh, 120, 255, cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Alle zusammenhängenden Konturen erkennen
    cnts = imutils.grab_contours(cnts)                                                    #
    
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        tl = (x, y)
        br = (x+w, y+h)
        mid = (x+w//2,y+h//2)
        cv2.circle(frame, mid, 10, (255,0,0), 2)
        #cv2.circle(frame, mid, 50, (255,255,0), 2)
        
        if mid_old is None:
            mid_old = mid
        
        distance = np.linalg.norm(np.array(mid_old)-np.array(mid))
        #print(mid_old, mid, distance)
        
        if distance < 100:
            if not flag == 1:
                p_lst.append(mid)
            else:
                p_lst2.append(mid)
        else:
            flag = 1
            
        mid_old = mid
        
    cv2.polylines(frame, [np.array(p_lst)], 0, (0,0,255), 3)
    cv2.polylines(frame, [np.array(p_lst2)], 0, (255,0,0), 3)
    
    frame_old = gray   #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen
    
#------------------------ Anzeigen ------------------------#

    cv2.circle(frame, center, rotor_radius, (0, 255, 255), 2)          # Kreise einzeichnen
    cv2.circle(frame, center, hub_circle_radius, (0, 0, 0), 2)         #
    cv2.circle(frame, center, 2, (255, 0, 255), 2)                     #
    
    cv2.imshow("Frame", frame)
    #cv2.imshow("line", line_image)
    #cv2.imshow("Thresh", thresh)
    #cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
    if key == ord("p"):
        time.sleep(5)
        
cv2.destroyAllWindows()
vs.release()

In [73]:
#------------------------ Imports ------------------------#
import imutils
import cv2
import numpy as np
import time
#------------------------ Videos festlegen ------------------------#
Video = 3
Videos = {
            1:["Simulation1",(917,518),170,20,13],
            2:["3D/Simulation1_Front",(962,548),200,20,48],
            3:["3D/Simulation2_Front",(962,548),200,20,48],
            4:["Kosmos1",(790,400),450,50,0],
            5:["Kosmos2",(790,400),450,50,0]
         }

#------------------------ Videoparameter festlegen ------------------------#
vs = cv2.VideoCapture(Videos[Video][0]+".mp4")
center = Videos[Video][1]
rotor_radius = Videos[Video][2]
hub_circle_radius = Videos[Video][3]
img_count = Videos[Video][4]

#------------------------ Kreis fürs Rotor ausblenden ------------------------#
frames_per_rotation = 180
theta = np.linspace(0, -2*np.pi, num=frames_per_rotation) #180 Werte zwischen 0 und 2*pi
x = rotor_radius*np.sin(theta) # x-Koordinaten nach Einheitskreis
y = rotor_radius*np.cos(theta) # y-Koordinaten nach Einheitskreis
rotor_positions = [(int(x[i])+center[0],int(y[i])+center[1]) for i in range(frames_per_rotation)]

#------------------------ Variablen ------------------------#
frame_old = None
birds = {0:[],1:[],2:[],3:[],4:[]}
#------------------------ Algorithmus ------------------------#
for i in range(0,int(vs.get(cv2.CAP_PROP_FRAME_COUNT))):
    img_count = (img_count+1)%frames_per_rotation            # Counter hochzählen (maximal 180)
    ret, frame = vs.read()                                   # Bild aus Video einlesen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)           # Bild zu grau konvertieren
    gray = cv2.GaussianBlur(gray, (21, 21), 0)               # Bild blurren
        
    if frame_old is None:                                          # Erstes Bild initialisieren
        frame_old = gray                                           #
        continue                                             #
    
    line_image = np.zeros(frame.shape[:-1])                  # Neues leeres Bild mit gleicher Größe erstellen (Für spätere Berechnungen)
    for i in range(0,frames_per_rotation,frames_per_rotation//3):
        pos = rotor_positions[(img_count+i)%frames_per_rotation]
        cv2.line(line_image, pos, center, (255,255,255), 65)              # Angewinkelte Linie auf leeres Bild einzeichnen
        #cv2.line(frame, pos, center, (255,255,255), 65)                  # Diese Linie auch auf frame anzeigen (optional)
    
    
    frameDelta = cv2.absdiff(frame_old, gray)                                                   # Differenz (Delta) zwischen altem und neuem Bild berechnen
    thresh = cv2.threshold(frameDelta, 10, 255, cv2.THRESH_BINARY)[1]                      # Delta threshholde
    thresh = cv2.dilate(thresh, None, iterations=4)                                       #
    thresh = thresh - np.uint8(line_image)
    thresh = cv2.threshold(thresh, 120, 255, cv2.THRESH_BINARY)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)    # Alle zusammenhängenden Konturen erkennen
    cnts = imutils.grab_contours(cnts)                                                    #
    
    current_mids = []                                      # Neue leere Liste für die Mittelpunkte
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)
        mid = np.array((x+w//2,y+h//2))
        cv2.circle(frame, mid, 10, (255,0,0), 2)
        cv2.circle(frame, mid, 50, (255,255,0), 2)
        
        current_mids.append(mid)                          # Alle Konturmittelpunkte werden der neuen Liste hinzugefügt
    
    for i in current_mids:
        d=[np.linalg.norm(i-j) for j in old_mids if list(j)!=list(i)] # Distanzen von Punkt i aus dem aktuellen Frame zu allen anderen Punkten aus dem letzten Frame
        
        # Ab hier brainfuck:
        # Punkt i muss dem Vogel zugeordnet werden, der min(d) hat.
        # Die folgenden 5 Zeilen sind noch Bullshit :-)
        if d and min(d) < 100:
            if len(d) == 1:
                birds[1].append(i)
            if len(d) == 2:
                birds[np.argmin(d)].append(i)
        
                  
    old_mids = current_mids
    
    
    
    
    cv2.polylines(frame, [np.array(birds[0])], 0, (0,0,255), 3)
    cv2.polylines(frame, [np.array(birds[1])], 0, (0,255,0), 3)
    cv2.polylines(frame, [np.array(birds[2])], 0, (255,0,0), 3)
    frame_old = gray   #Setze old auf aktuelles Bild, um es im nächsten Durchlauf zu vergleichen
    
#------------------------ Anzeigen ------------------------#

    cv2.circle(frame, center, rotor_radius, (0, 255, 255), 2)          # Kreise einzeichnen
    cv2.circle(frame, center, hub_circle_radius, (0, 0, 0), 2)         #
    cv2.circle(frame, center, 2, (255, 0, 255), 2)                     #
    
    cv2.imshow("Frame", frame)
    #cv2.imshow("line", line_image)
    #cv2.imshow("Thresh", thresh)
    #cv2.imshow("Frame Delta", frameDelta)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break
    if key == ord("p"):
        cv2.waitKey(-1)
        
cv2.destroyAllWindows()
vs.release()